# Assignment 3

## Week 8 - MapReduce

The exercises for this week has been executed using regular python scripts and not through IPython. Thus, the results you're seeing in the exercises below is pasted from the stdout of the terminal.

### Exercise 8.1

_Define and implement a MapReduce job to count the occurrences of each word in a text file. Document that it works with a small example._

In this exercise we are going to count the occurunces of each word in a text file. For this job, we've chosen the shakespeare corpus, since it is a corpus we are familiar with.

In [ ]:
r = RegexpTokenizer(u'\w+|[^\w\s\ufeff]+')

class MRWordCount(MRJob):
    def mapper(self, key, line):
        for word in r.tokenize(line):
            yield word.lower(), 1

    def combiner(self, word, c):
        yield word, sum(c)

    def reducer(self, key, vals):
        yield key, sum(vals)

Here is how the MapReduce job computes the word occurences:
1. The mapper tokenizes each line using `RegexpTokenizer` from nltk. 2. The mapper yields a tuple with the word as a key and 1 as a value.
2. The combiner then goes through all the tuples and sums the values for all the keys in all the lines.
3. Finally, the reducer sums all the values by the key and values. (This is different from the combiner that did in on a line-level)

The script is executed using a regular main method

In [ ]:
if __name__ == '__main__':
    MRWordCount.run()

To start the script we need to provide an input file as an argument. This can be done through the CLI.

`python test_mapreduce_word_count.py ../shakespeare.txt > job_output.txt`

The output of the MapReduce job is written to `job_output.txt`. Let's take a look into the file and see if everything looks like what we would expect. To do this, we shuffle the lines by using `gshuf` from `coreutils` (available for OS X through homebrew) and we can take the 10 first lines.

`gshuf text.txt | head -10`

```
"disdainful"    1
"promises"      1
"wash"  2
"wound" 4
"taxing"        1
"lovers"        6
"boughs"        3
"beast" 3
"goldsmiths"    1
"content"       7
```

### Exercise 8.2

Define and implement a MapReduce job that determines if a graph has an Euler tour (all vertices have even degree) where you can assume that the graph you get is connected. This file https://www.dropbox.com/s/usdi0wpsqm3jb7f/eulerGraphs.txt?dl=0 has 5 graphs – for each graph, the first line tells the number of nodes N and the number of edges E. The next E lines tells which two nodes are connected by an edge. Two nodes can be connected by multiple edges.

It is fine if you split the file into 5 different files. You do not need to keep the node and edge counts in the top of the file.

Document that it works using a small example.

This exercise will be concerned with finding out whether the graphs provided in the exercise description has an eularian path or has not. This should be a straight-forward task since an eulerian graph is a graph where all the nodes have a even degree.

The following MapReduce job decomposes this task into 3 `MRStep`s. Since this is a coherent class, we won't decompose the code, but we will describe the MapReduce pipeline below:
1. The first `MRStep` consists of a mapper that splits the edges into two tuples, each consisting of a node key and a count of 1.
The reducer, within the same `MRStep`, then counts the occurunces of each node in the edges (i.e. producing the degree).
2. The second `MRStep` then checks if the node has an even degree. If it does, a logic True value is produced.
3. the last `MRStep` checks if any node has an odd degree. If it has, there is no eulerian path in the graph. 

In [ ]:
class MRWordCount(MRJob):

    def steps(self):
        return [
            MRStep(mapper=self.mapper_1,
                   reducer=self.reducer_1),
            MRStep(reducer=self.reducer_2),
            MRStep(reducer=self.reducer_3)
        ]

    def mapper_1(self, key, line):
        e = line.split()
        yield e[0], 1
        yield e[1], 1

    def reducer_1(self, key, values):
        yield key, sum(values)

    def reducer_2(self, key, values):
        yield 'Euler graph?', values.next() % 2 == 0

    def reducer_3(self, key, values):
        yield 'Euler graph?', not False in list(values)

Executing the code is done through the terminal by running the following command for all the graphs:

`python test_mapreduce_euler_path.py graphs/graph1.txt`

```
Graph1 -- "Euler graph?"  true
Graph2 -- "Euler graph?"  false
Graph3 -- "Euler graph?"  false
Graph4 -- "Euler graph?"  false
Graph5 -- "Euler graph?"  false
```

The only graph that has an eulerian path, thus is the first graph `Graph1`.

### Exercise 8.3

Implement the MapReduce job from the lecture which finds common friends. To test it out, use the example from the slides and this one https://www.dropbox.com/s/ln0maf3q9xa08sf/facebook_combined.txt?dl=0 (note that for the Facebook file, you need to extend the job to convert from a list of edges to the format from the slides – do this with an additional map/reduce job).Document that it works using a small example.

### Exercise 8.4

Make a MapReduce job which counts the number of triangles in a graph. Use the following graph http://www.cise.ufl.edu/research/sparse/matrices/SNAP/roadNet-CA.html

Document that it works using a small example.

## Week 9 - Apache Spark

### Exercise 9.1

In this exercise we're going to use Apache Spark to make a simple word count. The file that we've choosed for this purpose is a some sample text of the Apache Spark Wikipedia page. The context looks like this:
```
Spark is a free and open-source software web application framework and domain-specific language written in Java. It is an alternative to other Java web application frameworks such as 
JAX-RS, Play framework and Spring MVC. It runs on an embedded Jetty web server by default, but can be configured to run on other webservers. Inspired by Sinatra, it does not follow the 
model–view–controller pattern used in other frameworks, such as Spring MVC. Instead, Spark is intended for "quickly creating web-applications in Java with minimal effort."[1]
Spark was created and open-sourced in 2011 by Per Wendel, and was completely rewritten for version 2 in 2014. The rewrite was hugely centered around the Java 8 lambda philosophy, so Java 7 is officially not supported in version 2 and above.
```

The text is persisted in a file called `sample_file.txt` placed in the working directory.

To get the file `sample_file.txt` we can use the `Pyspark` function `textFile()` which takes the filename as an argument and returns a RDD object (_Resilient Distributed Dataset_).

In [ ]:
f = sc.textFile('sample_file.txt')

The RDD Object has currently grouped the file line-by-line. Since we're going to count each word, we need to tokenize each sentence. Since this task is an embarrassingly parallel task (can be partitioned into as many processes as there are lines), we can apply a mapping function. However, since we want to produce multiple key-value pairs (tuples) for each sentence, we need to use `flatMap()`. The `flatMap()` function takes another function, as an argument, that returns a iterable and produces multiple tuples.

In [ ]:
# Tokenize lines
rdd = f.flatMap(lambda x: x.split())

Now that we've tokenized the sentences using a `flatMap` transform, we can make another `map` transform that produces a key-value tuple, which can be reduced by the tokenized word (i.e. the key).

In [ ]:
# Map each row
rdd = rdd.map(lambda x: (x,1))

In the last tranform step, we need to apply a `reduce` transform to the RDD object. Since this task involves multiple tuples, we can't consider it as being an embarrassingly parallel task. This `reduce` step is similar to the `fold` function we're familiar with from functional programming languages, where the variables `a` and `b` represent the current value pair and a variable that is kept in memory during the reduction.

In [ ]:
# Group by count
rdd = rdd.reduceByKey(lambda a,b: a+b)

Finally, we can call the `collect()` action, which executes the Apache spark job and returns the word count in a list of tuples.

In [ ]:
# Execute
rdd.collect()

```
[(u'and', 6),
 (u'Java', 4),
 (u'JAX-RS,', 1),
 (u'rewrite', 1),
 (u'is', 4),
 (u'supported', 1),
 (u'Jetty', 1),
 (u'Per', 1),
 (u'an', 2),
 (u'MVC.', 2),
 (u'as', 2),
 (u'webservers.', 1),
 (u'embedded', 1),
 (u'frameworks,', 1),
 (u'in', 6),
 (u'follow', 1),
 (u'alternative', 1),
 (u'web-applications', 1),
 (u'creating', 1),
 (u'Sinatra,', 1),
 (u'web', 3),
 (u'Play', 1),
 (u'effort."[1]', 1),
 (u'rewritten', 1),
 (u'for', 2),
 (u'application', 2),
 (u'default,', 1),
 (u'intended', 1),
 (u'Java.', 1),
 (u'frameworks', 1),
 (u'to', 2),
 (u'written', 1),
 (u'version', 2),
 (u'7', 1),
 (u'model\u2013view\u2013controller', 1),
 (u'8', 1),
 (u'Spark', 3),
 (u'officially', 1),
 (u'configured', 1),
 (u'minimal', 1),
 (u'can', 1),
 (u'be', 1),
 (u'such', 2),
 (u'used', 1),
 (u'run', 1),
 (u'around', 1),
 (u'centered', 1),
 (u'Inspired', 1),
 (u'open-sourced', 1),
 (u'free', 1),
 (u'it', 1),
 (u'Spring', 2),
 (u'framework', 2),
 (u'domain-specific', 1),
 (u'but', 1),
 (u'not', 2),
 (u'philosophy,', 1),
 (u'The', 1),
 (u'2014.', 1),
 (u'by', 3),
 (u'2011', 1),
 (u'a', 1),
 (u'on', 2),
 (u'runs', 1),
 (u'open-source', 1),
 (u'language', 1),
 (u'created', 1),
 (u'Wendel,', 1),
 (u'was', 3),
 (u'with', 1),
 (u'It', 2),
 (u'"quickly', 1),
 (u'server', 1),
 (u'above.', 1),
 (u'2', 2),
 (u'so', 1),
 (u'does', 1),
 (u'hugely', 1),
 (u'pattern', 1),
 (u'completely', 1),
 (u'the', 2),
 (u'software', 1),
 (u'other', 3),
 (u'Instead,', 1),
 (u'lambda', 1)]
 ```

### Exercise 9.2

In this exercise we are going to compute the euler tour of the given graphs and see if they contain an eulerian path. Since a eulerian graph cannot contain a node with an odd degree, we can just count the degrees for all the nodes, sum them and see if the x mod 2 is equal to zero.

Once again, we need to initialize the data. Prior to the loading, we've partitioned each graph into a different `.txt` document, making it easier to work with. We've also removed the total counts in the start of each graph, since they're not relevant for our task.

In [ ]:
f = sc.textFile('graphs/graph1.txt')

Similar to the previous exercise, we can use a `flatMap()` transform to split the edges into each node. Afterwards, we construct a key-value tuple for counting.

In [ ]:
rdd = f.flatMap(lambda x: x.split())
rdd = rdd.map(lambda x: (x,1))

To count the values by key, we make a `reduce` transform.

In [ ]:
rdd = rdd.reduceByKey(lambda a,b: a+b)

At this point we do not need the key (the node) anymore, but just the counts. Thus, we transform all those key-value pair into just a value.

In [ ]:
rdd = rdd.map(lambda x: x[1])

Now that we've a list of integers, we can sum them and see if mod 2 equals zero. In case it does, we hav a eulerian path in our graph.

In [ ]:
is_eularian = int(rdd.sum()) % 2 == 0

In [4]:
'The path is eularian' if is_eularian else 'This path is not eularian'

'The path is eularian'

To apply all this to all the graphs, we can compose everything we just did into a function that takes the filanme as an argument and returns whether the path was eularian or not:

In [ ]:
def is_graph_eularian(filename):
    f = sc.textFile(filename)
    rdd = f.flatMap(lambda x: x.split())
    rdd = rdd.map(lambda x: (x,1))
    rdd = rdd.reduceByKey(lambda a,b: a+b)
    rdd = rdd.map(lambda x: x[1] % 2 == 0)
    is_eularian = False not in rdd.collect()
    return 'The path is eularian' if is_eularian else 'This path is not eularian'

We can construct a `set` of all the filenames and then apply it to all graphs.

In [ ]:
filenames = ['graph1.txt', 'graph2.txt', 'graph3.txt', 'graph4.txt', 'graph5.txt']

In [ ]:
for filename in filenames:
    print filename[:-4] + ' -- ' + is_graph_eularian('graphs/'+filename)

```
graph1 -- The path is eularian
graph2 -- This path is not eularian
graph3 -- This path is not eularian
graph4 -- This path is not eularian
graph5 -- This path is not eularian
```

As it can be seen on the result, the 1st graph has an eularian path, meanwhile all the others do not. This result corresponds to the one we got when doing the same calculation using MapReduce.

### Exercise 9.3

In this exercise we're going to analyze wifi data. The frist thing we need to do, is to initilize the textfile into Spark.

In [ ]:
f = sc.textFile('wifi.data.txt')

We are now ready to construct and perform Spark queries on the initialized data set.

#### 10 Most observed networks

The first thing we need to do, is to get the 10 most observed networks by this particular mobile phone. To do this, we can use the MAC address of the router to uniquely identify each router. But since the MAC address does not make sense by itself, we can combine the `SSID` and `BSSID` in a tuple, and count their occurunces.

In [ ]:
a = f.map(lambda x: eval(x))
a = a.map(lambda x: ((x['ssid'], x['bssid']),1))

The code above enumerates all (`ssid`,`bssid`) tuples into another tuple, so they are ready to be counted. The code below then reduces the tuples in the `RDD` using `reduceByKey`. This function reduces the tuples by the 0th item in the tuple.

In [ ]:
a = a.reduceByKey(lambda a,b: a+b)

Now we have the result, but before showing it we would like to sort it by their counts in a descending order. Given a large distributed data set, this is usually a complicated task, but Spark makes it very easy:

In [ ]:
a = a.sortBy(lambda x: x[1], ascending=False)

We're ready to execute our Spark code, by using the `take()` function to get the head 10 tuples.

In [ ]:
a = a.sortBy(lambda x: x[1], ascending=False)

```
[((u'AirLink5GHz126C18', u'34:21:09:12:6c:1a'), 347),
 ((u'NETGEAR_1', u'00:24:b2:98:39:d2'), 338),
 ((u'AirLink126C18', u'34:21:09:12:6c:18'), 324),
 ((u'Housing People', u'e8:08:8b:c9:c1:79'), 318),
 ((u'Lausten_5GHz', u'44:94:fc:56:08:fb'), 315),
 ((u'Bronx', u'00:22:b0:b3:f2:ea'), 314),
 ((u'Playhouse', u'2c:b0:5d:ef:08:2b'), 272),
 ((u'Lausten', u'44:94:fc:56:ce:5e'), 240),
 ((u'Kaspers Wi-Fi-netv\xe6rk', u'28:cf:e9:84:a1:c3'), 211),
 ((u'Internet4realz', u'bc:ee:7b:55:1a:43'), 210)]
```

The `AirLink5GHz126C18` router is the most observed network by this particular mobile phone within a given time period.

### 10 most common wifi names

Wifi names often have common names, since people often keep the default SSID on their routers. In this part of the exercise we are going to find out what the most common of these Wifi names are (`SSID`).

Once again, we need to reinitialize our `rdd` variable `a`.

In [ ]:
a = f.map(lambda x: eval(x))

When the data has been reinitialized, we can group the tuples in the `RDD` by `SSID`.

In [ ]:
a = a.groupBy(lambda x: x['ssid'])

We are almost ready to count the wifi names. However, since the `SSID` is repeated for each log entry, we need to remove all duplicates for each router. We can do that by converting the `BSSID` (MAC address of router) into a set and counting the length of the set.

In [ ]:
a = a.map(lambda x: (x[0], len(set([v['bssid'] for v in x[1]]))))

We can then sort the `RDD` on the count in a descending order.

In [ ]:
a = a.sortBy(lambda x: x[1], ascending=False)

And finally taking the top 10 values.

In [ ]:
a.take(10)

```
[(u'', 15),
 (u'SIT-GUEST', 15),
 (u'SIT-BYOD', 13),
 (u'SIT-PROD', 13),
 (u'PDA-105', 11),
 (u'KNS-105', 11),
 (u'MED-105', 11),
 (u'KEA-PUBLIC', 6),
 (u'GST-105', 5),
 (u'wireless', 4)]
```

The result depicted above reveals that an empty `SSID` is the most common. This might be by routers that is hiding their SSID for security reasons. The router name `SIT-GUEST`, which is equally common as an empty `SSID`, might belong to a larger institute/firm that has multiple router hotspots, with the same names, accross their locations.

### 10 longest wifi names

The first thing we need, is to reinitilize the rdd. This time, we will only be in need of the distinct `SSID`s, so we start by removing all unecessary data from the tuples.

In this exercise we are interested in finding the longest wifi names (i.e. longest `SSID`).

In [ ]:
a = f.map(lambda x: eval(x)['ssid'])

Since our tuple only contains the `SSID`, we can remove all the duplicates by calling `distinct()`.

In [ ]:
a = a.distinct()

Counting the length of the individual wifi names is a embarrasingly parallel task, so we can use `map()` to count length using `len()` and construct a new tuple.

In [ ]:
a = a.map(lambda x: (x, len(x)))

Finally, we can sort the wifi names by `SSID` length in a descending order.

In [ ]:
a = a.sortBy(lambda x: x[1], ascending=False)
a.take(10)

```
[(u'HP-Print-43-Deskjet 3520 series', 31),
 (u'TeliaGatewayA4-B1-E9-2C-9E-CA', 29),
 (u'TeliaGateway08-76-FF-84-FF-8C', 29),
 (u'TeliaGateway9C-97-26-57-15-F9', 29),
 (u'TeliaGateway08-76-FF-46-3E-36', 29),
 (u'TeliaGateway9C-97-26-57-15-99', 29),
 (u'TeliaGateway08-76-FF-8A-EE-32', 29),
 (u'TeliaGateway08-76-FF-85-04-2F', 29),
 (u'TeliaGateway08-76-FF-9C-E0-82', 29),
 (u'Charlotte R.s Wi-Fi-netv\xe6rk', 27)]
 ```

From the  result above, we can see that `HP-Print-43-Deskjet 3520 series` is the longest Wifi `SSID` that has been observed by the mobile phone.

## Week 10 - Deep Learning

### Exercise 10.1